## 3. Building LSTM on GPU with PyTorch

### Model A: 1 Hidden Layer

2 things must live on the GPU
* model
* Variables

### Steps
1. Load Dataset
- Make Dataset iterable
- **Create model class**
- **Instantiate model class**
- Instantiate loss class
- Instantiate optimizer class
- **Train model!**

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [3]:
'''
Step 1: Load dataset
'''
train_dataset = dsets.MNIST(root="./data",
                            train=True,
                            transform=transforms.ToTensor(), 
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())
'''
Step 2: Make dataset iterable
'''
batch_size = 100
n_iters = 3000
num_epochs = int(n_iters / (len(train_dataset)/batch_size))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)
'''
Step 3: Create model class
'''
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        # Hidden Dimensions
        self.hidden_dim = hidden_dim
        
        # Number of Hidden Layers
        self.layer_dim = layer_dim
        
        # Building your LSTM
        # batch_first=True causes input/output tensors to be of shape
        # (batch_size, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        
        # Readout Layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Initialize hidden state with zeros
        #####################
        # USE GPU FOR MODEL #
        #####################
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
            # Initialize cell state (GPU)
            c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            # Initialize cell state (CPU)
            c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
        
        # 28 time steps
        out, (hn, cn) = self.lstm(x, (h0, c0))
        
        # Index hidden state of last time step
        # out.size() --> (100, 28, 100)
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states!
        out = self.fc(out[:, -1, :])
        # out.size() --> (100, 10)
        return out
'''
Step 4: Instantiate model class
'''

input_dim = 28
hidden_dim = 100
layer_dim = 3  #The only change from one layer to two layers is here!! 
output_dim = 10

model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)

#####################        
# USE GPU FOR MODEL #
#####################
if torch.cuda.is_available():
    model.cuda()

print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())
'''
Step 5: Instantiate Loss Class
'''
criterion = nn.CrossEntropyLoss()
'''
Step 6: Instantiate optimizer class
'''
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
'''
Step 7: Train the model
'''

# Number of steps to unroll
seq_dim = 28

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        #####################
        # USE GPU FOR MODEL #
        #####################
        if torch.cuda.is_available():
            images = Variable(images.view(-1, seq_dim, input_dim).cuda()) 
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1, seq_dim, input_dim)) 
            labels = Variable(labels)
        
        #Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        #Forward pass to get outputs / logits
        # outputs.size() --> (100, 10)
        outputs = model(images)
        
        #Calculate Loss: softmax --> Cross Entropy Loss
        loss = criterion(outputs, labels)
        
        #Get gradients w.r.t. parameters
        loss.backward()
        
        #Update parameters
        optimizer.step()
        
        iter += 1
        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0 
            total = 0
            #Iterate through the test dataset
            for images, labels in test_loader:
                # Load images to Torch Variable
                #####################
                # USE GPU FOR MODEL #
                #####################
                if torch.cuda.is_available():
                    images = Variable(images.view(-1, seq_dim, input_dim).cuda())
                else:
                    images = Variable(images.view(-1, seq_dim, input_dim))
                
                # Forward pass only to get outputs/logits
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                #####################
                # USE GPU FOR MODEL #
                #####################
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
                    
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

LSTMModel (
  (lstm): LSTM(28, 100, num_layers=3, batch_first=True)
  (fc): Linear (100 -> 10)
)
14
torch.Size([400, 28])
torch.Size([400, 100])
torch.Size([400])
torch.Size([400])
torch.Size([400, 100])
torch.Size([400, 100])
torch.Size([400])
torch.Size([400])
torch.Size([400, 100])
torch.Size([400, 100])
torch.Size([400])
torch.Size([400])
torch.Size([10, 100])
torch.Size([10])
Iteration: 500. Loss: 2.3015224933624268. Accuracy: 11.35
Iteration: 1000. Loss: 2.2988438606262207. Accuracy: 11.35
Iteration: 1500. Loss: 2.0118486881256104. Accuracy: 28.61
Iteration: 2000. Loss: 0.8705956935882568. Accuracy: 67.45
Iteration: 2500. Loss: 0.5202692151069641. Accuracy: 84.09
Iteration: 3000. Loss: 0.13889235258102417. Accuracy: 91.47


## Summary of LSTM RNN's
They are pretty great. Really, just go back and run this code when the battery is full and then re-run on GPU machine for comparison